<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/03_PDF_Summarizer_with_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create an agent that:
1. Loads a PDF file
2. Extracts and chunks the content
3. Summarizes each chunk intelligently
4. Produces a full summary of the document

# Install Required Libraries

In [1]:
!pip install fitz PyMuPDF transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.9/425.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00


# Define PDF Reader Tool

In [3]:
import pymupdf as fitz # PyMuPDF

def extract_text_from_pdf(path):
  doc = fitz.open(path)
  full_text = ""
  for page in doc:
    full_text += page.get_text()
  return full_text

# Download a sample PDF

In [4]:
!wget https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf -O dummy.pdf

--2025-08-30 17:08:27--  https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf
Resolving www.w3.org (www.w3.org)... 104.18.22.19, 104.18.23.19, 2606:4700::6812:1713, ...
Connecting to www.w3.org (www.w3.org)|104.18.22.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13264 (13K) [application/pdf]
Saving to: ‘dummy.pdf’

dummy.pdf           100%[===================>]  12.95K  --.-KB/s    in 0s      

2025-08-30 17:08:28 (83.1 MB/s) - ‘dummy.pdf’ saved [13264/13264]



# Chunk the extracted content

In [5]:
def chunk_text(text, chunk_size=500):
    """Chunks text into smaller pieces."""
    words = text.split()
    chunks = []
    current_chunk = []
    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# Load a summarization model

In [8]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


# Summarize Each Chunk

In [15]:
def summarize_chunks(chunks, summarizer):
    """Summarizes each chunk of text."""
    summaries = []
    for chunk in chunks:
        # Adjust max_length based on chunk length, ensuring a minimum
        max_len = min(150, max(30, len(chunk.split())))
        summary = summarizer(chunk, max_length=max_len, min_length=10, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries

# Example usage (assuming 'chunks' and 'summarizer' are available from previous steps)
# text = extract_text_from_pdf('dummy.pdf')
# chunks = chunk_text(text)
# chunk_summaries = summarize_chunks(chunks, summarizer)
# print(chunk_summaries)

# Assemble Final Summary

In [16]:
def summarize_pdf(path):
    print("Extracting PDF text...")
    full_text = extract_text_from_pdf(path)
    print("Chunking text...")
    chunks = chunk_text(full_text)
    print("Generating summaries...")
    summaries = summarize_chunks(chunks, summarizer) # Pass the summarizer object
    final_summary = "\n\n".join(summaries)
    return final_summary

# Run the Agent


In [17]:
pdf_path = "dummy.pdf"
summary = summarize_pdf(pdf_path)
print("\n📝 FINAL SUMMARY:\n")
print(summary)

Your max_length is set to 30, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


Extracting PDF text...
Chunking text...
Generating summaries...

📝 FINAL SUMMARY:

Dummy PDF file. Dummy PDF files. Dummies.
